In [1]:
import sys
sys.path.insert(0, "..")
import pickle
from gpbp_osm.layers import AdmArea
from gpbp_osm import visualisation
from gpbp_osm.distance import calculate_isopolygons_graph, calculate_isopolygons_Mapbox, population_served
from gpbp_osm.road_network import get_road_network_overpass

import osmnx as ox
import warnings
warnings.filterwarnings(action='ignore')

c:\Users\EiriniK\Documents\repos\Public-Infrastructure-Location-Optimiser\.venv\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
adm_area = AdmArea(country="Timor-Leste", level=0)

Retrieving data for Timor-Leste of granularity level 0
Extracting geometry for Timor-Leste


In [ ]:
adm_area.get_facilities(method="osm", tags={"building":"hospital"})
visualisation.plot_facilities(adm_area.fac_gdf)

In [ ]:
adm_area.get_population(method="world_pop")
visualisation.plot_population_heatmap(adm_area.pop_df)

In [ ]:
adm_area.compute_potential_fac(spacing=0.05)
visualisation.plot_facilities(adm_area.grid_gdf)

In [6]:
road_network = get_road_network_overpass(adm_area.geometry, network_type="driving", graph_type="networkx")

Building network


In [7]:
serve_dict_mapbox = population_served(
    adm_area.pop_df,
    adm_area.fac_gdf,
    "facilities",
    "length",
    2000,
    "driving",
    strategy = "mapbox",
    pop_resolution=4)